Reference [labml.ai](https://github.com/labmlai/annotated_deep_learning_paper_implementations/blob/940b3c01fc87c9487ad9478eec09f0167e177e2a/labml_nn/transformers/positional_encoding.py#L1)

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
%cd /content/drive/MyDrive/data_science_project/deeplearning_paper/transformer

/content/drive/MyDrive/data_science_project/deeplearning_paper/transformer


In [10]:
import math

import torch
import torch.nn as nn

from utils.positional_encoding import get_positional_encoding
from utils.mha import MultiHeadAttention

# Positional Encoding

## Fixed positional encoding

In [8]:
class EmbddingWithPositionalEncoding(nn.Module):
    """
    논문에 나온 방식, 고정된 positinal encoding
    """

    def __init__(self, d_model: int, n_vocab: int, max_len: int = 5000):
        super().__init__()
        self.linear = nn.Embedding(n_vocab, d_model)
        self.d_model = d_model
        self.register_buffer('positional_encodings', get_positional_encoding(d_model, max_len))

    def forward(self, x: torch.Tensor):
        pe = self.positional_encodings[:x.shape[0].reguires_grad_(False)]
        return self.linear(x) * math.sqrt(self.d_model) + pe

## Paremeterized positional encoding

In [9]:
class EmbddingsWithLearnedPositionalEncoding(nn.Module):
    """
    훈련 가능한 parameter로 positional encoding 수행
    """
    def __init__(self, d_model: int, n_vocab: int, max_len: int = 5000):
        super().__init__()
        self.linear = nn.Embedding(n_vocab, d_model)
        self.d_model = d_model
        self.positional_encodings = nn.Parameter(torch.zeros(max_len, 1, d_model), requires_grad=True) # (S, B, E)
    

    def forward(self, x: torch.Tensor):
        pe = self.positional_encodings[:x.shape[0]]
        return self.linear(x) * math.sqrt(self.d_model) + pe

# TransformerLayer

In [ ]:
from torch.nn.modules.activation import MultiheadAttention
class TransformerLayer(nn.Module):
    def __init__(self, *,
                 d_model: int, 
                 self_attn: MultiHeadAttention, # self attentionmodule
                 src_attn: MultiheadAttention = None, # source attention module (디코더일 때 사용)
                 feed_forward: FeedForward, # feedforward 네트워크
                 dropout_prob: float
                 ):
        
        super